In [17]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

# 设置全局随机种子
random_seed = 24
np.random.seed(random_seed)
random.seed(random_seed)

**经过preprocess.py，我们实现了对于原始csv文件的序列提取，后续需要进行的就是对于DNA序列的编码和随机森林模型的构建**

### **序列的编码方法**

#### **1. 使用`OrdinalEncoder`编码器对序列进行编码和模型构建**

##### 文件的读取和`OrdinalEncoder`的实现

In [18]:
def get_OrdinalEncoder_seq_and_result(filename):
    df = pd.read_csv(filename)
    whole_seqs = np.array(df['seq'])
    seperated_seqs = np.array([list(seq) for seq in whole_seqs])
    results = df['result'].to_list()
    results = np.array([1 if result == True else 0 for result in results])
    ordinal_encoder = OrdinalEncoder()
    ordinal_seq = ordinal_encoder.fit_transform(seperated_seqs)
    
    return ordinal_seq, results
'''
该函数的输出结果是一个维度为(n_samples, n_features)的向量和一个维度为(n_samples,)的向量
前一个向量表征输入DNA的特征，后一个是标签向量
在下方的代码实现具体输入文件的处理
'''
ordinal_DNA_train , ordinal_result_train = get_OrdinalEncoder_seq_and_result('K562_CTCF_train_seq_result.csv')
ordinal_DNA_test , ordinal_result_test = get_OrdinalEncoder_seq_and_result('K562_CTCF_test_seq_result.csv')
print(ordinal_DNA_train.shape)
print(ordinal_result_train.shape)

(8000, 1000)
(8000,)


##### 对输入特征的`PCA`降维和预测
**随机森林模型的调参过程，在预先训练中尝试了网格搜索`GridsearchCV`，但由于k折交叉训练的过程耗时太久，很多时候一小时都无法跑完一次网格搜索，因此这里我使用了手动调参数观察训练集预测结果的思路，在后面的过程中呈现的参数都是在手动参数调后确定的相对较优参数。**

In [19]:
# PCA过程
ordinal_pca = PCA(n_components=300, random_state= random_seed)
ordinal_DNA_train_pca = ordinal_pca.fit_transform(ordinal_DNA_train)
ordinal_DNA_test_pca = ordinal_pca.transform(ordinal_DNA_test)

# 模型建立
ordinal_clf_pca = RandomForestClassifier(n_estimators= 250, max_features= 1, max_depth= None, min_samples_split=2, random_state= random_seed)
ordinal_clf_pca.fit(ordinal_DNA_train_pca,ordinal_result_train)

# 训练集预测
ordinal_predict_pca = ordinal_clf_pca.predict(ordinal_DNA_test_pca)
ordinal_report_pca = classification_report(ordinal_result_test, ordinal_predict_pca)
print(ordinal_report_pca)

              precision    recall  f1-score   support

           0       0.62      0.03      0.05       979
           1       0.51      0.99      0.67      1021

    accuracy                           0.52      2000
   macro avg       0.57      0.51      0.36      2000
weighted avg       0.57      0.52      0.37      2000



**可以看到，对于降维后的结果，随机森林的建模非常畸形，对于1，即`True`类，其的召回率达到了0.99，与之相对的`False`类的召回率只有0.03，我们不难判断，该模型对于所有的序列都作出了为`True`的预测，因此实际上模型并没有做真正的预测，只是在投机取巧。尝试了几个不同的降维维度，结果都是一样的，我们可以猜测降维实际上使得碱基之间的上下文联系被忽视，因此是不够好的，对于后续的过程我们不考虑降维的方法。**

##### 原始输入特征的模型训练和预测

In [25]:
# 模型建立和训练
ordinal_clf = RandomForestClassifier(max_features=0.5, n_estimators=300, max_depth= None, min_samples_split=2, random_state= random_seed)
ordinal_clf.fit(ordinal_DNA_train,ordinal_result_train)

# 训练集预测
ordinal_predict = ordinal_clf.predict(ordinal_DNA_test)
ordinal_report = classification_report(ordinal_result_test, ordinal_predict)
print(ordinal_report)

# 尝试使用的GridSearchCV过程

# ordinal_clf = RandomForestClassifier()
# ordial_grid = {
#     'n_estimators':[200, 300],
#     'max_features':[0.3, 0.65, 1]
# }

# ordinal_search = GridSearchCV(ordinal_clf, ordial_grid, cv=5, scoring='accuracy', verbose= 3)
# ordinal_search.fit(ordinal_DNA_train, ordinal_result_train)
# ordinal_bestmodel = ordinal_search.best_estimator_
# print("最佳超参数组合: ", ordinal_search.best_params_)

# ordinal_predict = ordinal_bestmodel.predict(ordinal_DNA_test)
# ordinal_report = classification_report(ordinal_result_test, ordinal_predict)
# print(ordinal_report)

              precision    recall  f1-score   support

           0       0.63      0.46      0.53       979
           1       0.59      0.75      0.66      1021

    accuracy                           0.60      2000
   macro avg       0.61      0.60      0.59      2000
weighted avg       0.61      0.60      0.60      2000



**可以看到，这一次我们的模型是真正进行了预测，而非几乎全都选择某一类来“碰运气”，但是整体准确度还是只有60%左右，只能说具备一定的预测能力，还需要后续对于编码方式的进一步改进以提升模型准确率。**

#### **2. 使用`kmer`的统计来构建特征**

**由于结合基序通常跨度达到10-20个碱基，因此考虑碱基之间的联系是十分必要的，而`OrdinalEncoder`还是对于每个碱基独立编码，缺少碱基之间的相互联系，因此这里考虑序列分析中常见的`kmer`概念来帮助特征的构建，主要思路是统计特定长度k的DNA片段出现次数，特征的每一列是一个`kmer`，值对应其在序列中出现的次数，根据这个思路可以做到碱基之间的联系，主要思路是借助`CountVectorizer`模块进行序列的分析。该模块原本用于语义分割，但是可以用来自定义analyze函数进行序列处理**

##### 文件的读取和`kmer`词表的构建

In [26]:
# 定义获取kmers的函数
def get_kmers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# 定义处理文件的函数
def process_data(filename, vectorizer=None, is_train=True):
    df = pd.read_csv(filename)
    whole_seqs = np.array(df['seq'])
    results = df['result'].astype(int).to_numpy()
    
    if is_train:
        kmer_frequency_matrix = vectorizer.fit_transform(whole_seqs)
    else:
        kmer_frequency_matrix = vectorizer.transform(whole_seqs) # 对于训练数据只进行转换
    
    return kmer_frequency_matrix, results

def get_kmer_freq_and_result(train_filename, test_filename, k):
    vectorizer = CountVectorizer(analyzer=lambda x: get_kmers(x, k), lowercase=False)
    
    kmer_freq_train, train_results = process_data(train_filename, vectorizer, is_train=True)
    kmer_freq_test, test_results = process_data(test_filename, vectorizer, is_train=False)
    
    return kmer_freq_train, kmer_freq_test, train_results, test_results

kmer_DNA_train, kmer_DNA_test, kmer_result_train, kmer_result_test= get_kmer_freq_and_result('K562_CTCF_train_seq_result.csv','K562_CTCF_test_seq_result.csv', 5)
print(kmer_DNA_train.shape)
print(kmer_DNA_test.shape)
print(kmer_result_test.shape)
print(kmer_result_train.shape)

(8000, 1034)
(2000, 1034)
(2000,)
(8000,)


##### 模型的定义和训练

In [27]:
# 建立预测模型
kmer_clf = RandomForestClassifier(n_estimators= 300, max_depth= None, max_features= 0.3, min_samples_split= 2, random_state= random_seed)

kmer_clf.fit(kmer_DNA_train, kmer_result_train)

# 进行预测
kmer_predict = kmer_clf.predict(kmer_DNA_test)
kmer_report = classification_report(kmer_result_test, kmer_predict)
print(kmer_report)

# 
# kmer_clf = RandomForestClassifier(n_estimators= 200, max_depth= None, max_features= 0.3, min_samples_split= 2)
# kmer_grid = {
#     'n_estimators':[100, 200],
#     'max_features':[0.3, 0.5],
#     'max_depth': [10, 15, 20, None]
# }
# kmer_search = GridSearchCV(estimator=kmer_clf, param_grid=kmer_grid, cv=5, scoring='accuracy', verbose=1)
# kmer_search.fit(kmer_DNA_train, kmer_result_train)
# kmer_bestmodel = kmer_search.best_estimator_

# print("最佳超参数组合: ", kmer_search.best_params_)
# kmer_predict = kmer_bestmodel.predict(kmer_DNA_test)


              precision    recall  f1-score   support

           0       0.65      0.74      0.69       979
           1       0.71      0.62      0.66      1021

    accuracy                           0.68      2000
   macro avg       0.68      0.68      0.68      2000
weighted avg       0.68      0.68      0.68      2000



**可以看到效果明显好于直接进行碱基编码，达到了接近70%的准确率，并且是真正进行了预测，这说明考虑碱基之间的联系是优化模型的方法，但是本身随机森林模型只能接受`[n_samples, n_features]`的输入格式，因此真正将motif转化为编码存在困难，还有一些思路，如使用一个滑动窗，获取一个`window_size`中所有碱基的信息，比如将编码结果相加等，经过实验后发现这个方法的预测结果实际效果有限，可能是总和之后能够提取到的实际序列信息有限，并不能很好地表征。**  
  
  
**而对于某个范围的特征提取，则是卷积神经网络的专长范围，因此我也尝试了卷积神经网络的处理方法，得到了不错的结果，在CNN_predict.ipynb中呈现了这个过程。**